In [ ]:
import syft as sy
import pycolab

In [ ]:
duet = sy.launch_duet(loopback=True)

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 1 : Now STOP and run the Dungeon Agent notebook until the same checkpoint.

## Add Custom Python at a Secure Entrypoint

This functionality allows the 🗝 Root Key holding Dungeon Master 🧙🏼‍♂️ to write plain Python including custom classes and have that added to a remotely executable entrypoint for the Dungeon Agent 🎮. Simply annotate a `func` with `sendable` and an optional `return_type` and then run `func.send(duet)`. The `return_type` must be a valid type on the Dungeon Agents AST.

In [ ]:
# @sy.sendable(return_type="pycolab.engine.Engine")
# def make_game():
#     from pycolab import ascii_art
#     from pycolab import things
    
#     ART = ["1@"]
    
#     class RollingDrape(things.Drape):
#         def update(self, *args):
#             pass


#     class SlidingSprite(things.Sprite):
#         def __init__(self, *args):
#             self._visible = False
#             pass

#         def update(self, *args):
#             pass


#     return ascii_art.ascii_art_to_game(
#         ART,
#         what_lies_beneath=" ",
#         sprites={
#             "1": ascii_art.Partial(SlidingSprite, 0),
#         },
#         drapes={"@": RollingDrape},
#         z_order="1@",
#     )

In [ ]:
@sy.sendable(return_type="pycolab.engine.Engine")
def make_game():
    # third party
    import numpy as np
    from pycolab import ascii_art
    from pycolab import things

    HELLO_ART = [
        "                                    ",
        "  #   #  ### #    #     ###         ",
        "  #   # #    #    #    #   #        ",
        "  ##### #### #    #    #   #        ",
        "  #   # #    #    #    #   #        ",
        "  #   #  ###  ###  ###  ###         ",
        "                                    ",
        "     @   @  @@@   @@@  @    @@@@  1 ",
        "     @   @ @   @ @   @ @    @   @ 2 ",
        "     @ @ @ @   @ @@@@  @    @   @ 3 ",
        "     @ @ @ @   @ @   @ @    @   @   ",
        "      @@@   @@@  @   @  @@@ @@@@  4 ",
        "                                    ",
    ]

    class RollingDrape(things.Drape):
        """A Drape that just `np.roll`s the mask around either axis."""

        # There are four rolls to choose from: two shifts of size 1 along both axes.
        _ROLL_AXES = [0, 0, 1, 1]
        _ROLL_SHIFTS = [-1, 1, -1, 1]

        def update(self, actions, board, layers, backdrop, all_things, the_plot):
            del board, layers, backdrop, all_things  # unused

            if actions is None:
                return  # No work needed to make the first observation.
            if actions == 4:
                the_plot.terminate_episode()  # Action 4 means "quit".

            # If the player has chosen a motion action, use that action to index into
            # the set of four rolls.
            if actions < 4:
                rolled = np.roll(
                    self.curtain,  # Makes a copy, alas.
                    self._ROLL_SHIFTS[actions],
                    self._ROLL_AXES[actions],
                )
                np.copyto(self.curtain, rolled)
                the_plot.add_reward(1)  # Give ourselves a point for moving.

    class SlidingSprite(things.Sprite):
        """A Sprite that moves in diagonal directions."""

        # We have four mappings from actions to motions to choose from. The mappings
        # are arranged so that given any index i, then across all sets, the motion
        # that undoes motion i always has the same index j.
        _DX = ([-1, 1, -1, 1], [-1, 1, -1, 1], [1, -1, 1, -1], [1, -1, 1, -1])
        _DY = ([-1, 1, 1, -1], [1, -1, -1, 1], [1, -1, -1, 1], [-1, 1, 1, -1])

        def __init__(self, corner, position, character, direction_set):
            """Build a SlidingSprite.

            Args:
            corner: required argument for Sprite.
            position: required argument for Sprite.
            character: required argument for Sprite.
            direction_set: an integer in `[0,3]` that selects from any of four
                mappings from actions to (diagonal) motions.
            """
            super(SlidingSprite, self).__init__(corner, position, character)
            self._dx = self._DX[direction_set]
            self._dy = self._DY[direction_set]

        def update(self, actions, board, layers, backdrop, all_things, the_plot):
            del board, layers, backdrop, all_things, the_plot  # unused
            # Actions 0-3 are motion actions; the others we ignore.
            if actions is None or actions > 3:
                return
            new_col = (self._position.col + self._dx[actions]) % self.corner.col
            new_row = (self._position.row + self._dy[actions]) % self.corner.row
            self._position = self.Position(new_row, new_col)

    return ascii_art.ascii_art_to_game(
        HELLO_ART,
        what_lies_beneath=" ",
        sprites={
            "1": ascii_art.Partial(SlidingSprite, 0),
            "2": ascii_art.Partial(SlidingSprite, 1),
            "3": ascii_art.Partial(SlidingSprite, 2),
            "4": ascii_art.Partial(SlidingSprite, 3),
        },
        drapes={"@": RollingDrape},
        z_order="12@34",
    )

## Now send the code to the Duet Domain

In [ ]:
make_game.send(duet)

## Test It Locally
We can see it exists at `syft.sandbox.{func}` and run it if we like.

In [ ]:
# test the entrypoint locally
game = sy.sandbox.make_game()
print(type(game))

## Allow State Download

In [ ]:
duet.requests.add_handler(action="accept")

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 2 : Now STOP and run the Dungeon Agent 🎮 notebook until the same checkpoint.

In [ ]:
duet.store.pandas